In [1]:
#This Notebook is Contributed by PRUDHVI NARAYANA BANDARU -LOVE TOWARDS DATA
#Please let me know whether i did any mistakes ...you can mail me @ pn4235700@gmail.com

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-risk-analysis-dataset-real-world-data/Case_Data.csv


In [3]:
#Loan Risk analysis using Random Forest Classifier

#steps
'''
->DATA CLEANING AND PREPROCESSING
->FITTING THE RANDOM FOREST ALGORITHM TO THE DATA
->PREDICTING THE TEST RESULT
->TEST ACCURACY OF THE RESULT (PLOT CONFUSION MATRIX)
->VISULIZE THE TEST SET RESULTS(PLOTING CONTOUR PLOTS)
'''

'\n->DATA CLEANING AND PREPROCESSING\n->FITTING THE RANDOM FOREST ALGORITHM TO THE DATA\n->PREDICTING THE TEST RESULT\n->TEST ACCURACY OF THE RESULT (PLOT CONFUSION MATRIX)\n->VISULIZE THE TEST SET RESULTS(PLOTING CONTOUR PLOTS)\n'

In [4]:
#->DATA CLEANING AND PREPROCESSING STEP

#importing the essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs

#Reading the data from csv file
file_path='/kaggle/input/loan-risk-analysis-dataset-real-world-data/Case_Data.csv'
data=pd.read_csv(file_path)



#removing the tuples with null values(i used this preprocessing steps to decrease the difficulty)
love=data.drop(['Row ID','annual_inc_joint','dti_joint','Unnamed: 50','emp_title', 'emp_length', 'title', 'mths_since_last_delinq',
       'revol_util', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d',
       'mths_since_last_major_derog', '36months', '60months','sub_grade','application_type','home_ownership','term','verification_status','issue_d','loan_status','purpose','earliest_cr_line','initial_list_status'],axis=1)
# null_cols=love.columns[love.isna().any()]
# display(null_cols)

x=love.drop('grade',axis=1)
y=love['grade']
#in this step i just asked what are the null columns ..and i simply removed them 

# string_columns = love.select_dtypes(include=['object']).columns
# print(string_columns)
love

,id,member_id,loan_amnt,funded_amnt,int_rate,installment,grade,annual_inc,dti,delinq_2yrs,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,60516983,64537751,20000,20000,12.29,667.06,C,65000.00,20.72,0,...,0.00,0.0,0.0,0.0,0.00,0,0,0,52303,41000
1,60187139,64163931,11000,11000,12.69,369.00,C,40000.00,24.57,0,...,100.81,0.0,0.0,0.0,10059.00,0,0,332,175731,13100
2,60356453,64333218,7000,7000,9.99,225.84,B,32000.00,32.41,0,...,54.40,0.0,0.0,0.0,225.84,0,0,0,202012,16300
3,59955769,63900496,10000,10000,10.99,327.34,B,48000.00,30.98,0,...,79.37,0.0,0.0,0.0,327.34,0,0,0,108235,34750
4,58703693,62544456,9550,9550,19.99,354.87,E,32376.00,32.54,0,...,137.88,0.0,0.0,0.0,354.87,0,0,0,45492,14100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212994,54067695,57608436,20000,20000,13.33,677.07,C,120000.00,9.04,0,...,1267.56,0.0,0.0,0.0,700.00,0,0,574,21474,5400
212995,54414443,57995197,6000,6000,11.53,197.95,B,25000.00,2.21,0,...,295.77,0.0,0.0,0.0,197.95,0,0,0,2176,4200
212996,54474829,58055589,18000,18000,19.19,468.82,E,120000.00,7.76,0,...,1663.56,0.0,0.0,0.0,468.82,1,0,4404,8414,15300
212997,54354054,57934790,7050,7050,15.61,246.51,D,18614.27,21.53,1,...,513.41,0.0,0.0,0.0,246.51,0,0,0,159660,26800


In [5]:
#splitting the data into training and testing using built in train-test_split method within sklearn.model_selection
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [6]:
# # !pip install scikit-learn
# #feature scaling ->can be done using MinMaxScaler function within sklearn.preprocessing
# from sklearn.preprocessing import MinMaxScaler
# mms=MinMaxScaler()
# x_train=mms.fit_transform(x_train)->error coming


#Here my data is consisting of string data also so here im unable to preprocess  the data

In [7]:
#fitting the randomforest algorithm for training data
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(n_estimators=15,criterion='entropy')
classifier.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=15)

In [8]:
#predicting the model with the test data set
y_pred=classifier.predict(x_test)
y_pred

array(['C', 'A', 'C', ..., 'D', 'C', 'B'], dtype=object)

In [9]:
#creating the cofusion matrx that visually shows the prediction v/s actual value
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[ 9207     1     0     0     0     0     0]
 [    2 15654     6     0     0     0     0]
 [    1     0 15289     0     0     0     0]
 [    0     0     0  7440     6     0     0]
 [    1     0     0     4  4211    10     0]
 [    0     0     0     3    64  1107     2]
 [    0     0     0     0    14    44   184]]


In [10]:
#here in the above output you can see that the values or count notbelongs to diagonal are incorrect predictions


In [11]:
# for i in y_pred:
#     display(i)